In [1]:
# app/state.py

from __future__ import annotations
from typing import TypedDict, List, Dict, Any, Literal, Optional
from langgraph.graph import MessagesState


class PatentResult(TypedDict, total=False):
    app_num: str                  # 출원번호
    title: str                    # 발명의 명칭
    abstract: str                 # 초록 (요약본이면 더 좋음)
    ipc_codes: List[str]          # 주요 IPC 코드들
    score: float                  # 유사도 점수 (0~1)
    # 필요하면 여기에 더 필드 추가 가능 (예: applicant, year 등)


class PatentSearchRecord(TypedDict, total=False):
    id: str                       # "ps_1", "ps_2" 같은 식별자
    query: str                    # 검색에 사용한 정제된 질의 (키워드/문장)
    top_k: int                    # 요청한 결과 개수
    results: List[PatentResult]   # 검색 결과 리스트


class IPCResult(TypedDict, total=False):
    ipc_code: str                 # "G06T 7/00" 등
    title: str                    # IPC 제목
    description: str              # IPC 설명 (짧은 요약)
    score: float | None           # 추천일 때만 의미 있는 점수, 설명일 땐 None 가능


class IPCRequestRecord(TypedDict, total=False):
    id: str                       # "ipc_1", "ipc_2" ...
    mode: Literal["tech_to_ipc", "code_explain"]
    input_text: Optional[str]     # 기술 설명 (mode가 tech_to_ipc일 때)
    codes: List[str]              # 설명을 요청한 IPC 코드들
    top_k: int                    # 추천 개수 또는 검색 개수
    results: List[IPCResult]      # IPC 결과 리스트


class MetaState(TypedDict, total=False):
    turn_index: int               # 몇 번째 턴인지
    last_used_tools: List[str]    # 이번 턴에 어떤 툴 썼는지 (["search_patent", ...])
    session_id: Optional[str]     # 나중에 세션 추적용

In [2]:
class State(MessagesState):
    """
    LangGraph에서 사용할 전역 상태 정의.
    MessagesState를 상속하면 자동으로 `messages` 필드가 들어있음.
    """
    short_summary: Optional[str]
    patent_history: List[PatentSearchRecord]
    ipc_history: List[IPCRequestRecord]
    other_notes: Optional[str]
    meta: MetaState

# messages:
# LangGraph 기본 → user/assistant 전체 대화 저장

# short_summary:
# 오랫동안 대화하면 MemoryNode가 요약해서 여기 넣음

# patent_history:
# 여러 번의 특허 검색 기록
# 마지막은 patent_history[-1]로 접근

# ipc_history:
# 여러 번의 IPC 요청/설명 기록

# other_notes:
# “이번 턴에서 답해야 하는 기타 사항 요약” 정도로 사용 가능

# meta:
# 턴 인덱스, 최근 사용 툴, 세션 ID 등 운영/디버깅용

In [3]:
# app/nodes/add_user.py

from typing import Dict, Any
from langgraph.graph import MessagesState

def add_user_node(state: State, user_input: str) -> State:
    """
    새 user 발화를 state.messages에 추가하는 노드.
    LangGraph에선 보통 state만 인자로 받게 만들지만,
    여기선 개념을 보기 쉽게 user_input을 분리해서 표시.
    실제 graph에서는 state 안에 이미 user 메시지가 들어있는 형태로 호출할 수도 있음.
    """
    messages = state.get("messages", [])
    messages.append({"role": "user", "content": user_input})

    # 턴 카운트 증가
    meta = state.get("meta", {})
    meta["turn_index"] = meta.get("turn_index", 0) + 1

    return {
        **state,
        "messages": messages,
        "meta": meta,
    }

In [ ]:
# app/nodes/memory.py

from app.state import State

MAX_MESSAGES = 30  # 예시: 너무 길어지면 앞부분부터 줄이기

def memory_node(state: State) -> State:
    """
    v1: messages 길이가 너무 길어지면 앞부분 일부를 잘라내는 단순 버전.
    v2에서는 LLM을 사용해서 short_summary를 생성하도록 확장할 예정.
    """
    messages = state.get("messages", [])

    if len(messages) > MAX_MESSAGES:
        # 나중엔 여기서 LLM으로 요약해서 short_summary에 넣고,
        # 오래된 메시지는 더 과감하게 줄이는 방향으로 확장 가능.
        trimmed = messages[-MAX_MESSAGES:]
        return {
            **state,
            "messages": trimmed,
        }

    return state

In [6]:
# ============================================================
# 3. 테스트 시나리오
#    - State 초기화
#    - 여러 번 add_user_node + memory_node 호출
#    - messages 길이와 meta.turn_index 확인
# ============================================================

def init_state() -> State:
    """처음 한 번만 쓸 초기 상태 생성 함수."""
    return {
        "messages": [],
        "short_summary": None,
        "patent_history": [],
        "ipc_history": [],
        "other_notes": None,
        "meta": {
            "turn_index": 0,
            "last_used_tools": [],
            "session_id": None,
        },
    }

# 실제 테스트 실행
state = init_state()

print("=== 초기 State ===")
print("messages 개수:", len(state["messages"]))
print("turn_index:", state["meta"]["turn_index"])
print()

# 35번 정도 사용자 발화가 들어온다고 가정하고 테스트
for i in range(35):
    user_text = f"테스트용 사용자 발화 {i+1}번째 입니다."
    state = add_user_node(state, user_text)
    state = memory_node(state)

print("=== 35번 턴 이후 State ===")
print("messages 개수:", len(state["messages"]))   # MAX_MESSAGES(30)로 잘 잘렸는지 확인
print("turn_index:", state["meta"]["turn_index"])  # 35여야 함
print()

# 마지막 몇 개 메시지 내용 확인
print("=== 마지막 5개 메시지 내용 ===")
for msg in state["messages"][-5:]:
    print(f"- ({msg['role']}) {msg['content']}")

=== 초기 State ===
messages 개수: 0
turn_index: 0

=== 35번 턴 이후 State ===
messages 개수: 30
turn_index: 35

=== 마지막 5개 메시지 내용 ===
- (user) 테스트용 사용자 발화 31번째 입니다.
- (user) 테스트용 사용자 발화 32번째 입니다.
- (user) 테스트용 사용자 발화 33번째 입니다.
- (user) 테스트용 사용자 발화 34번째 입니다.
- (user) 테스트용 사용자 발화 35번째 입니다.


In [7]:
print("=== 초기 상태 필드들 확인 ===")
print("short_summary:", state.get("short_summary"))
print("patent_history:", state.get("patent_history"))
print("ipc_history:", state.get("ipc_history"))
print("other_notes:", state.get("other_notes"))
print("meta:", state.get("meta"))
print()

# 👉 특허 검색 결과가 한 번 들어왔다고 가정하고, mock 데이터 추가해보기
mock_patent_record = {
    "id": "ps_1",
    "query": "온도 표시 4인치 디스플레이 보온병",
    "top_k": 5,
    "results": [
        {
            "app_num": "10-2015-0000001",
            "title": "온도 표시 기능을 포함한 보온병",
            "abstract": "보온 기능과 함께 내부 온도를 디스플레이로 출력하는 발명...",
            "ipc_codes": ["G01K 7/00", "G06F 3/01"],
            "score": 0.91,
        }
    ],
}

# 👉 IPC 추천 결과도 한 번 들어왔다고 가정
mock_ipc_record = {
    "id": "ipc_1",
    "mode": "tech_to_ipc",
    "input_text": "온도를 표시하는 4인치 디스플레이가 달린 보온병",
    "codes": [],
    "top_k": 3,
    "results": [
        {
            "ipc_code": "G01K 7/00",
            "title": "측온 또는 온도 표시 장치",
            "description": "온도 측정 및 표시와 관련된 기술을 포함...",
            "score": 0.95,
        },
        {
            "ipc_code": "G06F 3/01",
            "title": "디스플레이 장치와 관련된 데이터 처리",
            "description": "디스플레이와 연계된 정보 처리 장치...",
            "score": 0.88,
        },
    ],
}

# state에 mock 기록 추가
pat_hist = state.get("patent_history", [])
ipc_hist = state.get("ipc_history", [])

pat_hist.append(mock_patent_record)
ipc_hist.append(mock_ipc_record)

state["patent_history"] = pat_hist
state["ipc_history"] = ipc_hist

print("=== mock 데이터 추가 후 상태 ===")
print("patent_history 길이:", len(state["patent_history"]))
print("ipc_history 길이:", len(state["ipc_history"]))
print()

print("patent_history[0]['query']:", state["patent_history"][0]["query"])
print("ipc_history[0]['results'][0]['ipc_code']:", state["ipc_history"][0]["results"][0]["ipc_code"])


=== 초기 상태 필드들 확인 ===
short_summary: None
patent_history: []
ipc_history: []
other_notes: None
meta: {'turn_index': 35, 'last_used_tools': [], 'session_id': None}

=== mock 데이터 추가 후 상태 ===
patent_history 길이: 1
ipc_history 길이: 1

patent_history[0]['query']: 온도 표시 4인치 디스플레이 보온병
ipc_history[0]['results'][0]['ipc_code']: G01K 7/00


In [8]:
import json
from typing import List, Dict, Any, Optional
from openai import OpenAI
import chromadb
from chromadb.config import Settings
from dotenv import load_dotenv
import os

load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

# OpenAI 클라이언트
client = OpenAI(api_key=OPENAI_API_KEY)

# 이미 위쪽에 client = OpenAI(...) 가 있다고 가정하고 작성


def decide_max_keywords(text: str) -> int:
    """
    입력 텍스트 길이에 따라 자동으로 키워드 개수를 결정하는 간단한 휴리스틱.
    - 300자 미만: 3개
    - 300 ~ 1500자: 5개
    - 1500자 이상: 7개
    """
    length = len(text)

    if length < 300:
        return 3
    elif length < 1500:
        return 5
    else:
        return 7


def tool_extract_core_keywords(
    tech_text: str,
    max_keywords: Optional[int] = None
) -> Dict[str, Any]:
    """
    LangGraph에서 'tool'로 쓸 수 있도록 설계한 키워드 추출 함수.

    입력:
        tech_text: 특허 기술 설명 (초록, 청구항 등)
        max_keywords: 키워드 최대 개수 (None이면 길이에 따라 자동 결정)

    출력:
        {
          "keywords": ["...", "...", ...],
          "used_max_keywords": 실제로 적용된 max_keywords 값 (int)
        }
    """

    # max_keywords 자동 결정
    if max_keywords is None:
        max_keywords = decide_max_keywords(tech_text)

    prompt = f"""
    너는 특허 및 IPC 검색을 위한 키워드 추출기야.

    아래 기술 설명을 읽고, 이 발명의 **핵심 기술 포인트**를 최대 {max_keywords}개까지 뽑아줘.
    **핵심 기술 포인트**는 이 발명이 기존 특허나 IPC와 유사/중복되는지를 판단하기 위해
    검색에 사용할 핵심적인 키워드 또는 짧은 구문이야.

    규칙:
    - "전자 장치", "터치 스크린 디스플레이", "무선 통신 회로", "센서", "메모리"처럼
      거의 모든 전자기기에 공통적으로 들어가는 일반적인 구성 요소 이름만 단독으로 쓰지 마.
    - 대신, 이 발명에서 **특히 중요한 동작 흐름/조건/조합**을 담아서 써.
      예: "결제 직전/직후 위치 데이터 + POI 서버 전송", "환경 센서 기반 결제 로그 축적"
    - 각 키워드는 1줄에 하나씩만 쓰고,
      설명 문장 대신 **벡터 검색에 쓸 수 있는 짧은 구문** 형태로 써.
    - 가능하다면, 이 기술이 적용되는 **도메인(예: 공장 생산 라인, 의료 영상, 자율주행 등)**이
      드러나도록 키워드를 작성해줘.

    기술 설명:
    \"\"\" 
    {tech_text}
    \"\"\"


    출력 형식(중요):
    - 반드시 JSON 배열 형태만 출력해.
    - 절대로 ``` 같은 마크다운 코드블록을 쓰지 마.
    - 'json'이라는 단어도 쓰지 마.
    - 예시: [
        "결제 직전 위치 데이터 + POI 서버 전송",
        "무선 결제 단말과 직접 연결된 위치 기반 결제",
        "환경 센서 기반 결제 로그 축적",
        "사용자 위치/환경 정보를 이용한 결제 기록 시스템"
      ]
    """

    res = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}]
    )

    raw = res.choices[0].message.content.strip()

    # JSON 형식 파싱 시도
    try:
        keywords = json.loads(raw)
        # 혹시 문자열 하나만 오거나 이상한 형식이면 보정
        if isinstance(keywords, str):
            keywords = [keywords]
        elif not isinstance(keywords, list):
            keywords = [str(keywords)]
    except json.JSONDecodeError:
        # JSON 실패 시 줄 단위 분리로 폴백
        keywords = [
            line.strip("-• ").strip()
            for line in raw.split("\n")
            if line.strip()
        ]

    # 공백 제거 및 빈 값 제거
    cleaned = [str(k).strip() for k in keywords if str(k).strip()]

    # max_keywords 개수 제한
    cleaned = cleaned[:max_keywords]

    return {
        "keywords": cleaned,
        "used_max_keywords": max_keywords,
    }

In [10]:
test_text = """
온도를 표시하는 4인치 디스플레이가 달린 보온병에 대한 비슷한 특허가 있는지 상위 5개를 찾아줘. 아 그리고 이용자와의 거리를 판단해서 자동으로 휘거나 평면으로 바뀌는 모니터가 있는지도 상위 10개를 검색해줘. 그리고 각각의 ipc코드는 뭘로해야할지 추천해주라. 아 마지막으로 이 특허들을 출원하는데 비용은 얼마나 들까?
"""

result = tool_extract_core_keywords(test_text)

print("used_max_keywords:", result["used_max_keywords"])
print("keywords:")
for k in result["keywords"]:
    print("-", k)

used_max_keywords: 3
keywords:
- 이용자 거리 판단에 따른 자동 변형 모니터
- 디스플레이 부착 보온병 온도 표시
- 4인치 디스플레이 온도 표시 시스템


In [ ]:
# ==== tools/patent_and_ipc_tools_stub.py 같은 느낌으로 써도 되고
# 일단 ipynb 한 파일에 붙여도 됨 ====

from __future__ import annotations
from typing import List, Optional, TypedDict, Literal
from uuid import uuid4

# 이미 app/state.py 에 있다고 가정 (없으면 같이 둬도 됨)
# from app.state import PatentResult, PatentSearchRecord, IPCResult, IPCRequestRecord


# -----------------------------
# 1) 특허 검색 툴 스텁
# -----------------------------

def tool_search_patent(
    query_text: str,
    top_k: int = 5,
    exclude_app_nums: Optional[List[str]] = None,
) -> PatentSearchRecord:
    """
    [스텁 버전]
    - 나중에 벡터 DB(특허 28,000건)와 연결될 툴의 인터페이스 정의.
    - 지금은 DB가 없으므로, 구조만 맞는 가짜 결과를 반환한다.

    파라미터:
    - query_text: 특허 검색에 사용할 질의 텍스트
    - top_k: 상위 몇 개 결과를 원하나요?
    - exclude_app_nums: 제외하고 싶은 출원번호 리스트 (follow-up 시 활용 예정)

    반환:
    - PatentSearchRecord 타입(dict) 한 개
    """

    # 나중에 여기서:
    # 1) query_text -> 임베딩
    # 2) 벡터 DB에서 cosine similarity로 top_k 검색
    # 3) exclude_app_nums 있으면 필터링
    # 4) PatentResult 리스트로 변환
    # 이런 식으로 구현하게 될 예정.

    fake_results: List[PatentResult] = []  # TODO: DB 붙으면 실제 결과 채우기

    record: PatentSearchRecord = {
        "id": f"ps_{uuid4().hex[:8]}",
        "query": query_text,
        "top_k": top_k,
        "results": fake_results,
    }

    return record


# -----------------------------
# 2) 기술 설명 → IPC 추천 툴 스텁
# -----------------------------

def tool_search_ipc_for_tech(
    tech_text: str,
    top_k: int = 5,
) -> IPCRequestRecord:
    """
    [스텁 버전]
    - 기술 설명(초록, 청구항, 키워드 등)을 바탕으로
      유사한 IPC 코드를 추천하는 용도.

    나중에:
    - tech_text -> 임베딩
    - IPC 임베딩 DB에서 유사한 코드 top_k 검색
    - IPCResult 리스트로 반환

    파라미터:
    - tech_text: IPC를 추천받고 싶은 기술 설명
    - top_k: 추천받고 싶은 IPC 코드 개수

    반환:
    - IPCRequestRecord (mode = 'tech_to_ipc')
    """

    fake_results: List[IPCResult] = []  # TODO: IPC DB 붙으면 실제 결과 채우기

    record: IPCRequestRecord = {
        "id": f"ipc_{uuid4().hex[:8]}",
        "mode": "tech_to_ipc",
        "input_text": tech_text,
        "codes": [],          # 추천 모드이므로 입력 코드 없음
        "top_k": top_k,
        "results": fake_results,
    }

    return record


# -----------------------------
# 3) IPC 코드 설명 툴 스텁
# -----------------------------

def tool_describe_ipc_codes(
    ipc_codes: List[str],
) -> IPCRequestRecord:
    """
    [스텁 버전]
    - 주어진 IPC 코드 리스트에 대해
      IPC DB에서 제목/설명을 찾아오는 용도.

    나중에:
    - 각 code에 대해 DB lookup
    - title/description을 IPCResult로 구성

    파라미터:
    - ipc_codes: 설명이 필요한 IPC 코드들 (예: ["G06T", "A61B"])

    반환:
    - IPCRequestRecord (mode = 'code_explain')
    """

    fake_results: List[IPCResult] = []  # TODO: IPC DB 붙으면 채우기

    record: IPCRequestRecord = {
        "id": f"ipc_{uuid4().hex[:8]}",
        "mode": "code_explain",
        "input_text": None,
        "codes": ipc_codes,
        "top_k": len(ipc_codes),  # 설명이니까 일반적으로 코드 개수만큼
        "results": fake_results,
    }

    return record


# -----------------------------
# 4) (옵션) OTHER용 메모 헬퍼
# -----------------------------
# 실제 LangGraph 상에서는 OTHER는 보통 "툴"보다는
# 그냥 LLM이 직접 답하는 영역이라 별도 툴이 꼭 필요하진 않음.
# 그래도 state.other_notes 같은 데 정리해서 기록해두고 싶다면,
# 이런 헬퍼를 쓸 수 있음.


class OtherNoteRecord(TypedDict, total=False):
    id: str
    summary: str      # 사용자의 기타 질문 요약
    detail: str       # 조금 더 긴 설명이나 메모


def tool_other_note(summary: str, detail: str = "") -> OtherNoteRecord:
    """
    [선택적 스텁]
    - DB를 쓰진 않지만, '기타(OTHER)' 요청에 대한 요약 메모를
      구조화해서 남기고 싶을 때 사용할 수 있는 헬퍼.

    파라미터:
    - summary: 한 줄 요약
    - detail: 필요하다면 좀 더 긴 설명

    반환:
    - OtherNoteRecord
    """
    record: OtherNoteRecord = {
        "id": f"other_{uuid4().hex[:8]}",
        "summary": summary,
        "detail": detail,
    }
    return record
